In [1]:
import warnings
from ipywidgets import Button, Layout
warnings.filterwarnings("ignore")
import pandas as pd
import matplotlib.patches as mpatches
from pprint import pprint
import numpy as np
from collections import defaultdict
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
import matplotlib.dates as mdates
from IPython.display import display, Markdown
import ipywidgets as widgets
import math
from ipywidgets import interact, interact_manual
from IPython.display import set_matplotlib_formats

pd.set_option('float_format', '{:.2f}'.format)
# plt.rcParams.update({'font.size': 28})

In [6]:
# Constants ----- #
BUCKET = 'CIC_IDS_2017_Friday_no_delta'

In [7]:
# Functions ----- #
def mprint(s):
    display(Markdown(s))
    

def describe_mtimeseries(key, mtimeserie):
    mprint(f"### {key}")
    # Data description ..... #
    start = min(mtimeserie.index)
    end = max(mtimeserie.index)
    mprint(f"**Time range**: {start} **/** {end}")
    mprint(f"**Total data range:** {end-start}")
    mprint(f"**Samples shown**: {len(mtimeserie)}")
    # Scaling ..... #
    scaled_ts = mtimeserie - mtimeserie.min()
    scaled_ts /= (mtimeserie.max()-mtimeserie.min())+1e-7
    
    # Plotting clean data ..... #
    empty_cols = []
    for idx, c in enumerate(mtimeserie.columns):
        if (scaled_ts[c] == 0).all(): 
            empty_cols.append(c)
            continue
        plt.figure()
        scaled_ts[c].plot(label=c, figsize=(8, 4))
        plt.gcf().suptitle(f"{key}")
        plt.legend()
        plt.show()
    for c in empty_cols:
        mprint(f"**<span style='color: red'>Empty series:</span> {c}**")
    
    mprint(f"<br>{'-'*50}<br>{'-'*50}<br>")

# Plotting

In [8]:
df = pd.read_pickle(f'../dataset/CIC-IDS-2017-dframes/{BUCKET}.pkl')

idxs = df.index.droplevel(2).unique()
devices = [f"{host} ({cat})" for cat, host in idxs]
devices.sort()
cols = list(df.columns)
cols.sort()

In [9]:
%matplotlib inline
set_matplotlib_formats('svg')

cols = list(df.columns)
cols.sort()

@interact
def show_articles_more_than(device=devices, 
                            selected_columns=widgets.SelectMultiple(
                                options=cols,
                                value=cols[:1],
                                description='Features',
                                layout=Layout(width='400px')
                            ),
                            showall=widgets.Checkbox(value=False, description='Show all'),
                            trange=widgets.FloatSlider(min=.05, max=1., step=.05),
                            offset=widgets.FloatSlider(min=.0, max=1., step=.05)):
    split = device.split(" ")
    columns = list(selected_columns)
    host = split[0].strip()
    category = " ".join(split[1:]).replace("(", "").replace(")", "").strip()
    
    host_ts = df.loc[category, host]
    ts_len = len(host_ts)
    start_idx = int(ts_len * offset)
    end_idx = min(start_idx + int(ts_len * trange), ts_len)
    
    selected_features = cols if showall else columns
    describe_mtimeseries(device.strip(), host_ts.iloc[start_idx:end_idx][selected_features])

interactive(children=(Dropdown(description='device', options=('192.168.10.1 (unknown device class)', '192.168.…

ModuleNotFoundError: No module named 'ntopng_constants'